# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [25]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets

In [26]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [27]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [28]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""

       # YOUR CODE HERE!
    housing_units_mean=sfo_data[['housing_units']].groupby([sfo_data.index])['housing_units'].mean()
    housing_units_min=housing_units_mean.min()
    housing_units_max=housing_units_mean.max()
    housing_units_std=housing_units_mean.std()
    lim_min=housing_units_min-housing_units_std
    lim_min 
    lim_max=housing_units_max+housing_units_std
    lim_max

    return housing_units_mean.hvplot.bar(title='Housing Units in San Francisco from 2010 to 2016', height=400, width=600,ylim=(lim_min,lim_max))
   
  

In [29]:
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    sfo_avg_price_rent_df = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby([sfo_data.index])['sale_price_sqr_foot','gross_rent'].mean()
    avg_gross_rent=sfo_avg_price_rent_df.reset_index().hvplot(x='year', y='sale_price_sqr_foot', title='Average Price per SqFt by Year',color='purple')
    
    return avg_gross_rent

In [30]:
def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    sfo_avg_price_rent_df = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby([sfo_data.index])['sale_price_sqr_foot','gross_rent'].mean()
    average_sales_price=sfo_avg_price_rent_df.reset_index().hvplot(x='year', y='gross_rent', title='Average Monthly Rent',color='red')

    return average_sales_price

In [31]:
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    sfo_data_new=sfo_data.groupby([sfo_data.index,"neighborhood"]).mean().reset_index()
    average_price_by_neighborhood=sfo_data_new.hvplot(x='year', y=['sale_price_sqr_foot'], groupby="neighborhood",ylabel='Avg. Sale Price per sqft')
    return average_price_by_neighborhood

In [37]:
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    # YOUR CODE HERE!
    top10_neighborhood=sfo_data.groupby(['neighborhood'])['sale_price_sqr_foot', 'housing_units', 'gross_rent'].mean()
    top10_neighborhood.sort_values(['sale_price_sqr_foot'],ascending=False,inplace=True)
    top10_expensive_neighborhood=top10_neighborhood.reset_index().head(10)
    return top10_expensive_neighborhood.hvplot.bar(x='neighborhood',y='sale_price_sqr_foot',ylabel='Avg. Sale Price per Square Foot', xlabel='Neighborhood')

In [38]:
def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!
    sfo_data_new=sfo_data.groupby([sfo_data.index,"neighborhood"]).mean().reset_index()
    return sfo_data_new.hvplot(x='year', y=['sale_price_sqr_foot'], groupby="neighborhood",ylabel='Avg. Sale Price per sqft')
    

In [39]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!


def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!


def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [40]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
welcome_column = pn.Column("## This dashboard presents the analysis of sale prices, gross rents based on the neighborhoods from 2010 to 2016", neighborhood_map())
market_analysis = pn.Column("## Yearly Market Analysis", housing_units_per_year(), average_gross_rent(), average_sales_price())
neighborhood_analysis = pn.Column("## Neighborhood Analysis",  average_price_by_neighborhood(), top_most_expensive_neighborhoods(), most_expensive_neighborhoods_rent_sales())

# Create a tab layout for the dashboard
# YOUR CODE HERE!
dashboard = pn.Tabs(
    (
        "Welcome",
        welcome_column
    ),
    (
        "Market Analysis",
        market_analysis
    ),
    (
        "Neighborhood Analysis",
        neighborhood_analysis
    )
)
   

In [41]:
# Create the dashboard
# YOUR CODE HERE!
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Str(None)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Bars)
        [3] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
housing_units_per_year()

In [ ]:
average_gross_rent()

In [ ]:
average_sales_price()

In [ ]:
average_price_by_neighborhood()

In [ ]:
top_most_expensive_neighborhoods()

In [ ]:
most_expensive_neighborhoods_rent_sales()

In [ ]:
#neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()